## 原生态xgboost的使用形式

### 安装xgboost教程
+ https://blog.csdn.net/weixin_40802676/article/details/89317332
### 官方文档 
+ https://xgboost.readthedocs.io/en/latest/build.html
### 扩展包
+ https://www.lfd.uci.edu/~gohlke/pythonlibs/
+ 查看本机python的版本匹配哪些文件

import pip._internal

print(pip._internal.pep425tags.get_supported())

In [1]:
import xgboost as xgb

### 名词收集
1. numpy  https://www.runoob.com/numpy/numpy-tutorial.html
2. pandas  https://morvanzhou.github.io/tutorials/data-manipulation/np-pd/
        https://blog.csdn.net/weixin_44489066/article/details/89494395
        https://www.jb51.net/article/163606.htm
3. xgboost
        https://www.jianshu.com/p/7e0e2d66b3d4
4. sklearn
        https://www.jianshu.com/p/6ada34655862
5. DataFrame

6. train_test_split
      https://www.cnblogs.com/bonelee/p/8036024.html

## 导入包及数据集

In [2]:
# 从sklearn 调入所需要的包
from sklearn import datasets
from sklearn.model_selection import train_test_split #数据分隔出训练集和验证集
import xgboost as xgb
import numpy as np
import pandas as pd
#导入精度和召回
from sklearn.metrics import precision_score, recall_score
#导入鸢尾花数据
iris = datasets.load_iris()
data = iris.data    #numpy 格式
label = iris.target
print(pd.DataFrame(data).head())    # 转化为DataFrame格式
print(pd.DataFrame(label).head())
data1 = pd.DataFrame(data)
## 花萼长宽花瓣长宽
data1.columns = ['sepal_l','sepal_w','petal_l','petal_w']
print(data1.head())
label1 = pd.DataFrame(label)
label1.columns=['label']
print(label1.head())
#注意这里data label顺序是一致的，千万别打乱

     0    1    2    3
0  5.1  3.5  1.4  0.2
1  4.9  3.0  1.4  0.2
2  4.7  3.2  1.3  0.2
3  4.6  3.1  1.5  0.2
4  5.0  3.6  1.4  0.2
   0
0  0
1  0
2  0
3  0
4  0
   sepal_l  sepal_w  petal_l  petal_w
0      5.1      3.5      1.4      0.2
1      4.9      3.0      1.4      0.2
2      4.7      3.2      1.3      0.2
3      4.6      3.1      1.5      0.2
4      5.0      3.6      1.4      0.2
   label
0      0
1      0
2      0
3      0
4      0


In [5]:
# print(iris)
type(iris)

sklearn.utils.Bunch

In [6]:
# 划分训练集和测试集
train_x, test_x, train_y, test_y = train_test_split(data1.values, label1.values, test_size=0.3, random_state=42)
print("训练集长度:", len(train_x))
print("测试集长度:", len(test_x))

训练集长度: 105
测试集长度: 45


In [14]:
# print(train_x)
# type(train_x)
# type(data1.values)
# print(data1.values)
type(data1)
# print(data1)

pandas.core.frame.DataFrame

In [21]:
test_data
# print(test_data)

In [ ]:
label1.label.value_counts()

1. xgb.DMatrix
        https://blog.csdn.net/u010159842/article/details/78053669
        https://blog.csdn.net/sb19931201/article/details/52577592

In [24]:
# 转换为DMatrix数据格式       #xgb矩阵赋值
# train_data = xgb.DMatrix(train_x, label=train_y)
test_data = xgb.DMatrix(test_x, label=test_y)
# 设置参数   https://blog.csdn.net/sb19931201/article/details/52577592
###multi：softmax是使用softmax后产生的分类结果，而multi:softprob是输出的概率矩阵。

xgb_params = {
    'eta': 0.3, #学习率
    'silent': True,  # 输出运行讯息
    'objective': 'multi:softprob',  # 使用多分类生成概率矩阵格式'multi:softmax',multi:softprob
    'num_class': 3,  # 共有几个类别
    'max_depth': 3  # 深度
}
num_round = 20  # 跑的步数    # 迭代次数

# 模型训练
model = xgb.train(xgb_params,xgb.DMatrix(train_x, label=train_y), num_round)

# LZY保存模型
model.save_model('C:/Users/Lee/Downloads/Homeworks/kaggle_lessons/lesson_1/开课吧讲课-数据竞赛-20191020/xgb.model') # 用于存储训练出的模型
# 模型预测
test_pre = model.predict(test_data)  #应该为test_x  ？？？

print(test_pre[:5])

# 选择表示最高概率的列（每一行最大值）
test_pre_1 = np.asarray([np.argmax(row) for row in test_pre])   # numpy格式
print("test的预测结果:",test_pre_1)

# 模型评估
print('验证集精准率：',precision_score(test_y, test_pre_1, average='macro'))  # https://blog.csdn.net/hfutdog/article/details/88085878
print('验证集召回率：',recall_score(test_y, test_pre_1, average='macro'))     # https://www.cnblogs.com/techengin/p/8962024.html

[[0.00650657 0.96226174 0.03123166]
 [0.970643   0.02533227 0.00402478]
 [0.0033913  0.00692109 0.9896876 ]
 [0.00654362 0.9677424  0.02571394]
 [0.00615641 0.9104776  0.08336602]]
test的预测结果: [1 0 2 1 1 0 1 2 1 1 2 0 0 0 0 1 2 1 1 2 0 2 0 2 2 2 2 2 0 0 0 0 1 0 0 2 1
 0 0 0 2 1 1 0 0]
验证集精准率： 1.0
验证集召回率： 1.0


解释： 第一行样本，选第二列（即1），预测为类别1；
     第二行样本，选第一列（即0），预测为类别0；
     。。。。。。

## Sklearn接口形式使用Xgboost

1. XGBClassifier
           https://www.cnblogs.com/wanglei5205/p/8579244.html
           
           https://blog.csdn.net/RR7970/article/details/96139104
           
           https://www.programcreek.com/python/example/99824/xgboost.XGBClassifier

In [25]:
from xgboost import XGBClassifier
model = XGBClassifier(
        learning_rate=0.01,#学习率
        n_estimators=3000,#步长
        max_depth=4,#深度
        objective='binary:logistic',
        seed=27
    )
model.fit(train_x,train_y)

# 预测
#输出预测结果
test_pre2 = model.predict(test_x)
print(test_pre2)
# 模型评估
print('验证集精准率：',precision_score(test_y, test_pre2, average='macro')) 
print('验证集召回率：',recall_score(test_y, test_pre2, average='macro'))   

# https://blog.csdn.net/hfutdog/article/details/88085878
# https://www.cnblogs.com/techengin/p/8962024.html

# 调参参数说明：
# https://www.cnblogs.com/wzdLY/p/9831282.html

# 原理及参数说明：
# https://blog.csdn.net/han_xiaoyang/article/details/52665396
# https://www.jianshu.com/p/a31091d5acbb    从参数空间转到了函数空间（观点新颖！！！）


H:\Anaconda3\anzhuang1\lib\site-packages\sklearn\preprocessing\label.py:219: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
H:\Anaconda3\anzhuang1\lib\site-packages\sklearn\preprocessing\label.py:252: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[1 0 2 1 1 0 1 2 1 1 2 0 0 0 0 1 2 1 1 2 0 2 0 2 2 2 2 2 0 0 0 0 1 0 0 2 1
 0 0 0 2 1 1 0 0]
验证集精准率： 1.0
验证集召回率： 1.0


## XGBOOST原理详解

+ https://www.cnblogs.com/kuangsyx/p/9043168.html